In [1]:
import sys
import numpy as np 
import matplotlib

# Basic version, trains on several samples

class Layer_Dense:
    def __init__(self, Number_of_inputs, Number_of_neurons):
        self.weights = np.abs(0.05 * np.random.randn(Number_of_inputs, Number_of_neurons))

        # This creates a matrix of weight factors connecting every neuron from the previous layer
        # to every neuron of the current layer
        # Initialisation of weights is best done with low values in order to prevent 'value explosion'
        self.biases = np.zeros((1, Number_of_neurons))
        # Each neuron also has a bias to vary
    def intermediate(self, inputs):
        self.output = np.dot(inputs, self.weights) + self.biases
        # This intermediate value is the input for the activation function

class Activation_ReLU:
    def forward(self, inputs):
        self.output = np.maximum(0, inputs)
    def derivative(self, inputs):
        self.diffoutput = np.ones(len(inputs))
        for a in range(0, len(inputs)):
            if inputs[a] > 0:
                self.diffoutput[a] = 1 #[1 for x in range(0, len(inputs))] 
            else:
                self.diffoutput[a] = 0 # [0 for x in range(0, len(inputs))] 
        
class Activation_Linear:
    def forward(self, inputs):
        self.output = np.array(inputs)  
    def derivative(self, inputs):
        self.diffoutput = [1 for x in range(0, len(inputs))] 

def Normalise_values(Value_list):
    Max_value = np.max(Value_list)
    for a in range(0,len(Value_list)):
        Value_list[a] = Value_list[a]/Max_value
    return np.array(Value_list)        

# Twice the samples, twice the fun
X = [[7.20218749e+01/7.20218749e+01, 1.14458687e-01], [32.55802208/7.20218749e+01,  0.46649978]]
y = [[1],[1]]

#X = Normalise_values([X[0][0],X[1][0]])
# print(X)
#y = Normalise_values(y)

Num_Inputs = 2
Num_Hidden_Neurons_1 = 5
Num_Outputs = 1

        
Hidden_Layer_1 = Layer_Dense(Num_Inputs,Num_Hidden_Neurons_1)
# Creates a hidden layer. The first value is the number of neurons in the previous layer (now input, so 1),
# the second value is the number of neurons in the current hidden layer (which can be any number)
Activation_Function_Hidden_Layer_1 = Activation_ReLU()
# Choose for all neurons of the layer a certain activation function

Final_Layer = Layer_Dense(Num_Hidden_Neurons_1,Num_Outputs) # Input: number of neurons hidden layer, number of outputs (=2)

Activation_Function_Final_Layer = Activation_Linear()


Hidden_Layer_1_Pseudo = Layer_Dense(Num_Inputs,Num_Hidden_Neurons_1)
# Creates a hidden layer. The first value is the number of neurons in the previous layer (now input, so 1),
# the second value is the number of neurons in the current hidden layer (which can be any number)
Activation_Function_Hidden_Layer_1_Pseudo = Activation_ReLU()
# Choose for all neurons of the layer a certain activation function

Final_Layer_Pseudo = Layer_Dense(Num_Hidden_Neurons_1,Num_Outputs) # Input: number of neurons hidden layer, number of outputs (=2)

Activation_Function_Final_Layer_Pseudo = Activation_Linear()

Learning_rate = 50
Learning_increaser = 1.01

def Pseudo_update(X):
    Hidden_Layer_1_Pseudo.intermediate(X)
    Activation_Function_Hidden_Layer_1_Pseudo.forward(Hidden_Layer_1_Pseudo.output)
    Output_Hidden_Layer_1_Pseudo = Activation_Function_Hidden_Layer_1_Pseudo.output
    Final_Layer_Pseudo.intermediate(Output_Hidden_Layer_1_Pseudo)
    Activation_Function_Final_Layer_Pseudo.forward(Final_Layer_Pseudo.output)
    Output_Final_Layer_Pseudo = Activation_Function_Final_Layer_Pseudo.output
    NN_y = [Output_Final_Layer_Pseudo[0][0]]
    Error = np.abs(((NN_y[0]*40)**2/((NN_y[0]*40)**2+X[0]**2))**(1/2)-X[1])
    return np.array([Error, NN_y])
    
def Security_update(X):
    Hidden_Layer_1_Pseudo.intermediate(X)
    Activation_Function_Hidden_Layer_1_Pseudo.forward(Hidden_Layer_1_Pseudo.output)
    Output_Hidden_Layer_1_Pseudo = Activation_Function_Hidden_Layer_1_Pseudo.output
    Final_Layer_Pseudo.intermediate(Output_Hidden_Layer_1_Pseudo)
    Activation_Function_Final_Layer_Pseudo.forward(Final_Layer_Pseudo.output)
    Output_Final_Layer_Pseudo = Activation_Function_Final_Layer_Pseudo.output

def dEdOut1(xobs,znn):
    return (1/2*xobs**2*(znn**2/(xobs**2+znn**2))**(1/2+1))/(znn**3) 

def NN_Training(X,y,Learning_rate,Learning_increaser,Error_array):
    Hidden_Layer_1.intermediate(X)
    # Create the input for the activation function by creating a property of the hidden layer

    Activation_Function_Hidden_Layer_1.forward(Hidden_Layer_1.output) # 'Hidden_Layer_1.output' is the values of the neurons
    # Calculate the (output) value of every neuron in layer 1 by performing the activation function

    Output_Hidden_Layer_1 = Activation_Function_Hidden_Layer_1.output # 'Activation_Function_Hidden_Layer_1.output' is the value of the output of the neurons

    Final_Layer.intermediate(Output_Hidden_Layer_1) # Which is the input for the next layer of neurons

    Activation_Function_Final_Layer.forward(Final_Layer.output)

    Output_Final_Layer = Activation_Function_Final_Layer.output
    
    # Make a copy for testing
    Hidden_Layer_1_Pseudo.intermediate(X)
    Activation_Function_Hidden_Layer_1_Pseudo.forward(Hidden_Layer_1_Pseudo.output) 
    Output_Hidden_Layer_1_Pseudo = Activation_Function_Hidden_Layer_1_Pseudo.output
    Final_Layer_Pseudo.intermediate(Output_Hidden_Layer_1_Pseudo) 
    Activation_Function_Final_Layer_Pseudo.forward(Final_Layer_Pseudo.output)
    Output_Final_Layer_Pseudo = Activation_Function_Final_Layer_Pseudo.output
    
    
    NN_y = [0 for x in range(0, len(X))]
    Error = [0 for x in range(0, len(X))]
    
    for x in range(0, len(X)):
        NN_y[x] = Output_Final_Layer[x]
        Error[x] = ((NN_y[x]*40)**2/((NN_y[x]*40)**2+(X[x][0]*7.20218749e+01)**2))**(1/2)-X[x][1]
        
#         print('Results show',((NN_y[x][0]*40)**2/((NN_y[x][0]*40)**2+X[x][0]**2))**(1/2))
        print('y(NN):',NN_y[x])
#         print('Desired output',np.array([y[x][0]/40,1]))
#         print('Error:',Error[x])
    
    ##
    ####### Input Layer -> Hidden Layer weights update
    ##
    
    for a in range(0,len(X[0])): # loop over the size of one input tuple; usually 2
        
        dEdG = [0 for x in range(0,len(X))]
        dGdOut = [0 for x in range(0,len(X))]
        dOutdIn = [0 for x in range(0,len(X))]
        dIndW = [0 for x in range(0,len(X))]
        dOutdInHid = [0 for x in range(0,len(X))]
        dInHiddW = [0 for x in range(0,len(X))]
        dEdW = [0 for x in range(0,len(X))]
        
        for b in range(0,len(X)): # Loop over the size of the input batch
        
            dEdG[b] = Error[b]  

            dGdOut[b] = dEdOut1(X[b][0]*7.20218749e+01,NN_y[b]*40)

            Activation_Function_Final_Layer.derivative(Final_Layer.output[b]) # value of the derivatives at the values of the output neurons
            # 1xn matrix, n=number of outputs

            dOutdIn[b] = Activation_Function_Final_Layer.diffoutput # Derivative of the output to the input; derivative of activation function
            #dOutdIn = Final_Layer.output[0] # Element 0 because this is a list of lists
            # 1xn matrix, n=number of outputs

            dIndW[b] = Hidden_Layer_1.output[b] # Element 0 because this is a list of lists

            Activation_Function_Hidden_Layer_1.derivative(Hidden_Layer_1.output[b]) # 1st neuron hidden layer value
            # This 'generates' derivatives in the layers, but doesn't retrieve them

            dOutdInHid[b] = Activation_Function_Hidden_Layer_1.diffoutput # Derivative of the output to the input; derivative of activation function
            # 1xn matrix, n=number of outputs

            dInHiddW[b] = X[b][a] # Derivative of the input to the weights, which is the input
            # 1xn matrix, n=number of outputs

            dEdW[b] = dEdG[b]*dGdOut[b]*dOutdIn[b]*dIndW[b]*dOutdInHid[b]*dInHiddW[b]
        
        dEdWAv = [0 for x in range(0,Num_Hidden_Neurons_1)]
        for x in range(0,Num_Hidden_Neurons_1):
            Tot_update = 0
            for y in range(0,len(X)): # Number of inputs in one batch
                Tot_update = Tot_update + dEdW[y][x]
            dEdWAv[x] = Tot_update/len(X)
        
        for b in range (0, Num_Hidden_Neurons_1):
            Old_weight = Hidden_Layer_1.weights[a][b]
            Hidden_Layer_1.weights[a][b] = Hidden_Layer_1.weights[a][b] - Learning_rate*dEdWAv[b]
            Hidden_Layer_1_Pseudo.weights[a][b] = Hidden_Layer_1.weights[a][b] - Learning_rate*dEdWAv[b]
            
            Old_error = [0 for x in range(0,len(X))] # Number of inputs in one batch
            New_error = [0 for x in range(0,len(X))] # Number of inputs in one batch
            NN_z_new = [0 for x in range(0,len(X))] # Number of inputs in one batch
            NN_q_new = [0 for x in range(0,len(X))] # Number of inputs in one batch
            
            for c in range(0,len(X)): # Number of inputs in one batch 
                Old_error[c] = Error[c]
                New_error[c] = Pseudo_update(X[c])[0]
                NN_z_new[c] = Pseudo_update(X[c])[1]
            
            Bad_improvement_bool = False
            
            if (np.any(np.array(NN_z_new)<0)): # Might be a too harsh requirement
                Hidden_Layer_1.weights[a][b] = Old_weight
                Hidden_Layer_1_Pseudo.weights[a][b] = Old_weight
                Bad_improvement_bool = True
            
            Security_update(X)
            
        if Bad_improvement_bool == True:
            Learning_rate = Learning_rate/Learning_increaser
#         else:
#             Learning_rate = Learning_rate*Learning_increaser
                
            
    
    ####### Hidden Layer -> Final Layer weights update
    
    for a in range(0,Num_Hidden_Neurons_1):
        
        dEdG = [0 for x in range(0,len(X))]
        dGdOut = [0 for x in range(0,len(X))]
        dOutdIn = [0 for x in range(0,len(X))]
        dIndW = [0 for x in range(0,len(X))]
        dOutdInHid = [0 for x in range(0,len(X))]
        dInHiddW = [0 for x in range(0,len(X))]
        dEdW = [0 for x in range(0,len(X))]
        
        for b in range(0,len(X)): # Loop over the number of inputs in the input batch

            dEdG[b] = Error[b]   

            dGdOut[b] = dEdOut1(X[b][0]*7.20218749e+01,NN_y[b]*40)

            Activation_Function_Final_Layer.derivative(Final_Layer.output[b]) # value of the derivatives at the values of the output neurons
            # This 'generates' derivatives in the layers, but doesn't retrieve them

            dOutdIn[b] = Activation_Function_Final_Layer.diffoutput # Derivative of the output to the input; derivative of activation function
            # 1xn matrix, n=number of outputs

            dIndW[b] = Hidden_Layer_1.output[b][a]

            dEdW[b] = dEdG[b]*dGdOut[b]*dOutdIn[b]*dIndW[b]
            
        dEdWAv = [0 for x in range(0,Num_Outputs)]
        for x in range(0,Num_Outputs):
            Tot_update = 0
            for y in range(0,len(X)): # Number of inputs in one batch
                Tot_update = Tot_update + dEdW[y][x]
            dEdWAv[x] = Tot_update/len(X)
        
        for b in range (0, Num_Outputs):
            Old_weight = Final_Layer.weights[a][b]
            Final_Layer.weights[a][b] = Final_Layer.weights[a][b] - Learning_rate*dEdWAv[b]
            Final_Layer_Pseudo.weights[a][b] = Final_Layer.weights[a][b] - Learning_rate*dEdWAv[b]
            
            Old_error = [0 for x in range(0,len(X))] # Number of inputs in one batch
            New_error = [0 for x in range(0,len(X))] # Number of inputs in one batch
            NN_y_new = [0 for x in range(0,len(X))] # Number of inputs in one batch
            
            for c in range(0,len(X)): # Number of inputs in one batch 
                Old_error[c] = Error[c]
                New_error[c] = Pseudo_update(X[c])[0]
                NN_z_new[c] = Pseudo_update(X[c])[1][0]
            
            Bad_improvement_bool = False
            
            if (np.any(np.array(NN_z_new)<0)): # Might be a too harsh requirement
                Final_Layer.weights[a][b] = Old_weight
                Final_Layer_Pseudo.weights[a][b] = Old_weight
                Bad_improvement_bool = True

            Security_update(X)    
                
        if Bad_improvement_bool == True:
            Learning_rate = Learning_rate/Learning_increaser
#         else:
#             Learning_rate = Learning_rate*Learning_increaser
                
            
    
    ##
    ####### Hidden layer bias update
    ##
    
    dEdG = [0 for x in range(0,len(X))]
    dGdOut = [0 for x in range(0,len(X))]
    dOutdIn = [0 for x in range(0,len(X))]
    dIndW = [0 for x in range(0,len(X))]
    dOutdInHid = [0 for x in range(0,len(X))]
    dInHiddB = [0 for x in range(0,len(X))]
    dEdB = [0 for x in range(0,len(X))]
    
    for b in range(0,len(X)): # Loop over the size of the input batch
        
        dEdG[b] = Error[b]   

        dGdOut[b] = dEdOut1(X[b][0]*7.20218749e+01,NN_y[b]*40)

        Activation_Function_Final_Layer.derivative(Final_Layer.output[b]) # value of the derivatives at the values of the output neurons
        # 1xn matrix, n=number of outputs

        dOutdIn[b] = Activation_Function_Final_Layer.diffoutput # Derivative of the output to the input; derivative of activation function
        #dOutdIn = Final_Layer.output[0] # Element 0 because this is a list of lists
        # 1xn matrix, n=number of outputs

        dIndW[b] = Hidden_Layer_1.output[b] # Element 0 because this is a list of lists

        Activation_Function_Hidden_Layer_1.derivative(Hidden_Layer_1.output[b]) # 1st neuron hidden layer value
        # This 'generates' derivatives in the layers, but doesn't retrieve them

        dOutdInHid[b] = Activation_Function_Hidden_Layer_1.diffoutput # Derivative of the output to the input; derivative of activation function
        # 1xn matrix, n=number of outputs

        dInHiddB[b] = np.ones(len(Hidden_Layer_1.output[b])) # Derivative of the input to the bias, which is an additive constant
        # 1xn matrix, n=number of outputs
        
        dEdB[b] = dEdG[b]*dGdOut[b]*dOutdIn[b]*dIndW[b]*dOutdInHid[b]*dInHiddB[b]
    
    dEdBAv = [0 for x in range(0,Num_Hidden_Neurons_1)]
    for x in range(0,Num_Hidden_Neurons_1):
        Tot_update = 0
        for y in range(0,len(X)): # Number of inputs in one batch
            Tot_update = Tot_update + dEdB[y][x]
        dEdBAv[x] = Tot_update/len(X)
    
    for a in range(0, Num_Hidden_Neurons_1):
        Old_bias = Hidden_Layer_1.biases[0][a]
        Hidden_Layer_1.biases[0][a] = Hidden_Layer_1.biases[0][a] - Learning_rate*dEdBAv[a]
        Hidden_Layer_1_Pseudo.biases[0][a] = Hidden_Layer_1.biases[0][a] - Learning_rate*dEdBAv[a]
        
        Old_error = [0 for x in range(0,len(X))] # Number of inputs in one batch
        New_error = [0 for x in range(0,len(X))] # Number of inputs in one batch
        NN_z_new = [0 for x in range(0,len(X))] # Number of inputs in one batch
        NN_q_new = [0 for x in range(0,len(X))] # Number of inputs in one batch
        
        for c in range(0,len(X)): # Number of inputs in one batch
            Old_error[c] = Error[c]
            New_error[c] = Pseudo_update(X[c])[0]
            NN_z_new[c] = Pseudo_update(X[c])[1][0]
        
        if (np.any(np.array(NN_z_new)<0)): # Might be a too harsh requirement
            Hidden_Layer_1.biases[0][a] = Old_bias
            Hidden_Layer_1_Pseudo.biases[0][a] = Old_bias
            Bad_improvement_bool = True
            
        Security_update(X)

    if Bad_improvement_bool == True:
        Learning_rate = Learning_rate/Learning_increaser
#     else:
#         Learning_rate = Learning_rate*Learning_increaser
    
    ##
    ######## Output layer bias update
    ##
    
    dEdG = [0 for x in range(0,len(X))]
    dGdOut = [0 for x in range(0,len(X))]
    dOutdIn = [0 for x in range(0,len(X))]
    dIndB = [0 for x in range(0,len(X))]
    dEdB = [0 for x in range(0,len(X))]
    
    for b in range(0,len(X)): # Loop over the number of inputs in the input batch
    
        dEdG[b] = Error[b]  

        dGdOut[b] = dEdOut1(X[b][0]*7.20218749e+01,NN_y[b]*40)

        Activation_Function_Final_Layer.derivative(Final_Layer.output[b]) # value of the derivatives at the values of the output neurons
        # This 'generates' derivatives in the layers, but doesn't retrieve them

        dOutdIn[b] = Activation_Function_Final_Layer.diffoutput # Derivative of the output to the input; derivative of activation function
        # 1xn matrix, n=number of outputs

        dIndB[b] = np.ones(len(Final_Layer.output[b])) # Derivative of the input to the bias, which is an additive constant
        # 1xn matrix, n=number of outputs

        dEdB[b] = dEdG[b]*dGdOut[b]*dOutdIn[b]*dIndB[b] # Total derivative of the energy to the biases
        # 1xn matrix, n=number of outputs
    
    dEdBAv = [0 for x in range(0,Num_Outputs)]
    for x in range(0,Num_Outputs):
        Tot_update = 0
        for y in range(0,len(X)): # Number of inputs in one batch
            Tot_update = Tot_update + dEdB[y][x]
        dEdBAv[x] = Tot_update/len(X)
    
    # Updating of all the biases at once using the learning rate and the gradient
    
    for a in range(0, Num_Outputs):
        Old_bias = Final_Layer.biases[0][a]

        Final_Layer.biases[0][a] = Final_Layer.biases[0][a] - Learning_rate*dEdBAv[a]
        Final_Layer_Pseudo.biases[0][a] = Final_Layer.biases[0][a] - Learning_rate*dEdBAv[a]
 
        Old_error = [0 for x in range(0,len(X))] # Number of inputs in one batch
        New_error = [0 for x in range(0,len(X))] # Number of inputs in one batch
        NN_z_new = [0 for x in range(0,len(X))] # Number of inputs in one batch
        NN_q_new = [0 for x in range(0,len(X))]
            
        for c in range(0,len(X)): # Number of inputs in one batch 
            Old_error[c] = Error[c]
            New_error[c] = Pseudo_update(X[c])[0]
            NN_z_new[c] = Pseudo_update(X[c])[1][0]
        
        if (np.any(np.array(NN_z_new)<0)): # Might be a too harsh requirement
            Final_Layer.biases[0][a] = Old_bias
            Final_Layer_Pseudo.biases[0][a] = Old_bias
            Bad_improvement_bool = True

        Security_update(X)
        
    if Bad_improvement_bool == True:
        Learning_rate = Learning_rate/Learning_increaser
#     else:
#         Learning_rate = Learning_rate*Learning_increaser

    print(' ')
    print('Learning_rate',Learning_rate)
    return Learning_rate
    
    #np.append(Error_array,Error_current_run)
    
    #return Error_array
    
Error_array = []
for a in range(0,5000):
    print('Iteration',a+1)
    Learning_rate = NN_Training(X,y,Learning_rate,Learning_increaser,Error_array)
    #Error_array = NN_Training(X[1],y[1],Learning_rate,Error_array)
    #for b in range(0,len(X)):
#         NN_Training(X[b],y[b],Learning_rate)

Iteration 1
y(NN): [0.00878156]
y(NN): [0.00723698]
 
Learning_rate 50
Iteration 2
y(NN): [0.20997156]
y(NN): [0.20766953]
 
Learning_rate 50
Iteration 3
y(NN): [0.29012777]
y(NN): [0.28745698]
 
Learning_rate 50
Iteration 4
y(NN): [0.32761859]
y(NN): [0.32482215]
 
Learning_rate 50
Iteration 5
y(NN): [0.34743324]
y(NN): [0.34463842]
 
Learning_rate 50
Iteration 6
y(NN): [0.35853297]
y(NN): [0.35580854]
 
Learning_rate 50
Iteration 7
y(NN): [0.3649335]
y(NN): [0.36231703]
 
Learning_rate 50
Iteration 8
y(NN): [0.36867451]
y(NN): [0.36618634]
 
Learning_rate 50
Iteration 9
y(NN): [0.3708685]
y(NN): [0.36851925]
 
Learning_rate 50
Iteration 10
y(NN): [0.37214811]
y(NN): [0.36994284]
 
Learning_rate 50
Iteration 11
y(NN): [0.37288216]
y(NN): [0.37082279]
 
Learning_rate 50
Iteration 12
y(NN): [0.37328892]
y(NN): [0.3713756]
 
Learning_rate 50
Iteration 13
y(NN): [0.37349878]
y(NN): [0.37173065]
 
Learning_rate 50
Iteration 14
y(NN): [0.37359009]
y(NN): [0.37196576]
 
Learning_rate 50
Iter

 
Learning_rate 50
Iteration 139
y(NN): [0.35980074]
y(NN): [0.37862953]
 
Learning_rate 50
Iteration 140
y(NN): [0.35959761]
y(NN): [0.37871259]
 
Learning_rate 50
Iteration 141
y(NN): [0.35939211]
y(NN): [0.37879652]
 
Learning_rate 50
Iteration 142
y(NN): [0.35918421]
y(NN): [0.37888131]
 
Learning_rate 50
Iteration 143
y(NN): [0.35897389]
y(NN): [0.37896698]
 
Learning_rate 50
Iteration 144
y(NN): [0.35876112]
y(NN): [0.37905351]
 
Learning_rate 50
Iteration 145
y(NN): [0.35854588]
y(NN): [0.37914092]
 
Learning_rate 50
Iteration 146
y(NN): [0.35832815]
y(NN): [0.37922921]
 
Learning_rate 50
Iteration 147
y(NN): [0.35810792]
y(NN): [0.37931837]
 
Learning_rate 50
Iteration 148
y(NN): [0.35788516]
y(NN): [0.37940842]
 
Learning_rate 50
Iteration 149
y(NN): [0.35765985]
y(NN): [0.37949936]
 
Learning_rate 50
Iteration 150
y(NN): [0.35743197]
y(NN): [0.37959119]
 
Learning_rate 50
Iteration 151
y(NN): [0.35720152]
y(NN): [0.37968392]
 
Learning_rate 50
Iteration 152
y(NN): [0.35696846

 
Learning_rate 50
Iteration 253
y(NN): [0.31876334]
y(NN): [0.39379997]
 
Learning_rate 50
Iteration 254
y(NN): [0.31825311]
y(NN): [0.39397492]
 
Learning_rate 50
Iteration 255
y(NN): [0.31774114]
y(NN): [0.39415025]
 
Learning_rate 50
Iteration 256
y(NN): [0.31722744]
y(NN): [0.39432595]
 
Learning_rate 50
Iteration 257
y(NN): [0.31671207]
y(NN): [0.39450199]
 
Learning_rate 50
Iteration 258
y(NN): [0.31619507]
y(NN): [0.39467837]
 
Learning_rate 50
Iteration 259
y(NN): [0.31567648]
y(NN): [0.39485506]
 
Learning_rate 50
Iteration 260
y(NN): [0.31515634]
y(NN): [0.39503207]
 
Learning_rate 50
Iteration 261
y(NN): [0.31463469]
y(NN): [0.39520936]
 
Learning_rate 50
Iteration 262
y(NN): [0.31411158]
y(NN): [0.39538694]
 
Learning_rate 50
Iteration 263
y(NN): [0.31358704]
y(NN): [0.39556479]
 
Learning_rate 50
Iteration 264
y(NN): [0.31306113]
y(NN): [0.39574289]
 
Learning_rate 50
Iteration 265
y(NN): [0.31253388]
y(NN): [0.39592122]
 
Learning_rate 50
Iteration 266
y(NN): [0.31200533

 
Learning_rate 50
Iteration 400
y(NN): [0.24644208]
y(NN): [0.41719579]
 
Learning_rate 50
Iteration 401
y(NN): [0.24608219]
y(NN): [0.4173081]
 
Learning_rate 50
Iteration 402
y(NN): [0.24572483]
y(NN): [0.41741959]
 
Learning_rate 50
Iteration 403
y(NN): [0.24536998]
y(NN): [0.41753028]
 
Learning_rate 50
Iteration 404
y(NN): [0.24501766]
y(NN): [0.41764017]
 
Learning_rate 50
Iteration 405
y(NN): [0.24466784]
y(NN): [0.41774925]
 
Learning_rate 50
Iteration 406
y(NN): [0.24432054]
y(NN): [0.41785753]
 
Learning_rate 50
Iteration 407
y(NN): [0.24397575]
y(NN): [0.41796501]
 
Learning_rate 50
Iteration 408
y(NN): [0.24363346]
y(NN): [0.4180717]
 
Learning_rate 50
Iteration 409
y(NN): [0.24329366]
y(NN): [0.41817758]
 
Learning_rate 50
Iteration 410
y(NN): [0.24295637]
y(NN): [0.41828268]
 
Learning_rate 50
Iteration 411
y(NN): [0.24262156]
y(NN): [0.41838698]
 
Learning_rate 50
Iteration 412
y(NN): [0.24228924]
y(NN): [0.41849049]
 
Learning_rate 50
Iteration 413
y(NN): [0.24195941]


 
Learning_rate 50
Iteration 522
y(NN): [0.21869695]
y(NN): [0.42580724]
 
Learning_rate 50
Iteration 523
y(NN): [0.21857581]
y(NN): [0.42584469]
 
Learning_rate 50
Iteration 524
y(NN): [0.2184559]
y(NN): [0.42588175]
 
Learning_rate 50
Iteration 525
y(NN): [0.21833722]
y(NN): [0.42591844]
 
Learning_rate 50
Iteration 526
y(NN): [0.21821975]
y(NN): [0.42595475]
 
Learning_rate 50
Iteration 527
y(NN): [0.21810347]
y(NN): [0.42599069]
 
Learning_rate 50
Iteration 528
y(NN): [0.21798839]
y(NN): [0.42602626]
 
Learning_rate 50
Iteration 529
y(NN): [0.21787449]
y(NN): [0.42606147]
 
Learning_rate 50
Iteration 530
y(NN): [0.21776175]
y(NN): [0.42609632]
 
Learning_rate 50
Iteration 531
y(NN): [0.21765017]
y(NN): [0.4261308]
 
Learning_rate 50
Iteration 532
y(NN): [0.21753973]
y(NN): [0.42616494]
 
Learning_rate 50
Iteration 533
y(NN): [0.21743044]
y(NN): [0.42619872]
 
Learning_rate 50
Iteration 534
y(NN): [0.21732226]
y(NN): [0.42623215]
 
Learning_rate 50
Iteration 535
y(NN): [0.21721521]


...

 
Learning_rate 50
Iteration 4407
y(NN): [0.2074516]
y(NN): [0.4292805]
 
Learning_rate 50
Iteration 4408
y(NN): [0.2074516]
y(NN): [0.4292805]
 
Learning_rate 50
Iteration 4409
y(NN): [0.2074516]
y(NN): [0.4292805]
 
Learning_rate 50
Iteration 4410
y(NN): [0.2074516]
y(NN): [0.4292805]
 
Learning_rate 50
Iteration 4411
y(NN): [0.2074516]
y(NN): [0.4292805]
 
Learning_rate 50
Iteration 4412
y(NN): [0.2074516]
y(NN): [0.4292805]
 
Learning_rate 50
Iteration 4413
y(NN): [0.2074516]
y(NN): [0.4292805]
 
Learning_rate 50
Iteration 4414
y(NN): [0.2074516]
y(NN): [0.4292805]
 
Learning_rate 50
Iteration 4415
y(NN): [0.2074516]
y(NN): [0.4292805]
 
Learning_rate 50
Iteration 4416
y(NN): [0.2074516]
y(NN): [0.4292805]
 
Learning_rate 50
Iteration 4417
y(NN): [0.2074516]
y(NN): [0.4292805]
 
Learning_rate 50
Iteration 4418
y(NN): [0.2074516]
y(NN): [0.4292805]
 
Learning_rate 50
Iteration 4419
y(NN): [0.2074516]
y(NN): [0.4292805]
 
Learning_rate 50
Iteration 4420
y(NN): [0.2074516]
y(NN): [0.4

 
Learning_rate 50
Iteration 4553
y(NN): [0.2074516]
y(NN): [0.4292805]
 
Learning_rate 50
Iteration 4554
y(NN): [0.2074516]
y(NN): [0.4292805]
 
Learning_rate 50
Iteration 4555
y(NN): [0.2074516]
y(NN): [0.4292805]
 
Learning_rate 50
Iteration 4556
y(NN): [0.2074516]
y(NN): [0.4292805]
 
Learning_rate 50
Iteration 4557
y(NN): [0.2074516]
y(NN): [0.4292805]
 
Learning_rate 50
Iteration 4558
y(NN): [0.2074516]
y(NN): [0.4292805]
 
Learning_rate 50
Iteration 4559
y(NN): [0.2074516]
y(NN): [0.4292805]
 
Learning_rate 50
Iteration 4560
y(NN): [0.2074516]
y(NN): [0.4292805]
 
Learning_rate 50
Iteration 4561
y(NN): [0.2074516]
y(NN): [0.4292805]
 
Learning_rate 50
Iteration 4562
y(NN): [0.2074516]
y(NN): [0.4292805]
 
Learning_rate 50
Iteration 4563
y(NN): [0.2074516]
y(NN): [0.4292805]
 
Learning_rate 50
Iteration 4564
y(NN): [0.2074516]
y(NN): [0.4292805]
 
Learning_rate 50
Iteration 4565
y(NN): [0.2074516]
y(NN): [0.4292805]
 
Learning_rate 50
Iteration 4566
y(NN): [0.2074516]
y(NN): [0.4

 
Learning_rate 50
Iteration 4704
y(NN): [0.2074516]
y(NN): [0.4292805]
 
Learning_rate 50
Iteration 4705
y(NN): [0.2074516]
y(NN): [0.4292805]
 
Learning_rate 50
Iteration 4706
y(NN): [0.2074516]
y(NN): [0.4292805]
 
Learning_rate 50
Iteration 4707
y(NN): [0.2074516]
y(NN): [0.4292805]
 
Learning_rate 50
Iteration 4708
y(NN): [0.2074516]
y(NN): [0.4292805]
 
Learning_rate 50
Iteration 4709
y(NN): [0.2074516]
y(NN): [0.4292805]
 
Learning_rate 50
Iteration 4710
y(NN): [0.2074516]
y(NN): [0.4292805]
 
Learning_rate 50
Iteration 4711
y(NN): [0.2074516]
y(NN): [0.4292805]
 
Learning_rate 50
Iteration 4712
y(NN): [0.2074516]
y(NN): [0.4292805]
 
Learning_rate 50
Iteration 4713
y(NN): [0.2074516]
y(NN): [0.4292805]
 
Learning_rate 50
Iteration 4714
y(NN): [0.2074516]
y(NN): [0.4292805]
 
Learning_rate 50
Iteration 4715
y(NN): [0.2074516]
y(NN): [0.4292805]
 
Learning_rate 50
Iteration 4716
y(NN): [0.2074516]
y(NN): [0.4292805]
 
Learning_rate 50
Iteration 4717
y(NN): [0.2074516]
y(NN): [0.4

 
Learning_rate 50
Iteration 4842
y(NN): [0.2074516]
y(NN): [0.4292805]
 
Learning_rate 50
Iteration 4843
y(NN): [0.2074516]
y(NN): [0.4292805]
 
Learning_rate 50
Iteration 4844
y(NN): [0.2074516]
y(NN): [0.4292805]
 
Learning_rate 50
Iteration 4845
y(NN): [0.2074516]
y(NN): [0.4292805]
 
Learning_rate 50
Iteration 4846
y(NN): [0.2074516]
y(NN): [0.4292805]
 
Learning_rate 50
Iteration 4847
y(NN): [0.2074516]
y(NN): [0.4292805]
 
Learning_rate 50
Iteration 4848
y(NN): [0.2074516]
y(NN): [0.4292805]
 
Learning_rate 50
Iteration 4849
y(NN): [0.2074516]
y(NN): [0.4292805]
 
Learning_rate 50
Iteration 4850
y(NN): [0.2074516]
y(NN): [0.4292805]
 
Learning_rate 50
Iteration 4851
y(NN): [0.2074516]
y(NN): [0.4292805]
 
Learning_rate 50
Iteration 4852
y(NN): [0.2074516]
y(NN): [0.4292805]
 
Learning_rate 50
Iteration 4853
y(NN): [0.2074516]
y(NN): [0.4292805]
 
Learning_rate 50
Iteration 4854
y(NN): [0.2074516]
y(NN): [0.4292805]
 
Learning_rate 50
Iteration 4855
y(NN): [0.2074516]
y(NN): [0.4

 
Learning_rate 50
Iteration 4966
y(NN): [0.2074516]
y(NN): [0.4292805]
 
Learning_rate 50
Iteration 4967
y(NN): [0.2074516]
y(NN): [0.4292805]
 
Learning_rate 50
Iteration 4968
y(NN): [0.2074516]
y(NN): [0.4292805]
 
Learning_rate 50
Iteration 4969
y(NN): [0.2074516]
y(NN): [0.4292805]
 
Learning_rate 50
Iteration 4970
y(NN): [0.2074516]
y(NN): [0.4292805]
 
Learning_rate 50
Iteration 4971
y(NN): [0.2074516]
y(NN): [0.4292805]
 
Learning_rate 50
Iteration 4972
y(NN): [0.2074516]
y(NN): [0.4292805]
 
Learning_rate 50
Iteration 4973
y(NN): [0.2074516]
y(NN): [0.4292805]
 
Learning_rate 50
Iteration 4974
y(NN): [0.2074516]
y(NN): [0.4292805]
 
Learning_rate 50
Iteration 4975
y(NN): [0.2074516]
y(NN): [0.4292805]
 
Learning_rate 50
Iteration 4976
y(NN): [0.2074516]
y(NN): [0.4292805]
 
Learning_rate 50
Iteration 4977
y(NN): [0.2074516]
y(NN): [0.4292805]
 
Learning_rate 50
Iteration 4978
y(NN): [0.2074516]
y(NN): [0.4292805]
 
Learning_rate 50
Iteration 4979
y(NN): [0.2074516]
y(NN): [0.4

In [19]:
class Station_point:
    def __init__(self, x, z, R, rho, q, m):
        G = 6.67408 *10**-11
        self.A = 4/3*np.pi*G*rho*R**3
        self.g = self.A*z**m/(x**2+z**2)**q
    def norm_g(self, g0):
        self.g0 = self.g/g0

R = 20
rho = 2500 
m = 1

z = 40
q = 3/2

Centre_x = 0
Centre_y = 0
p0 = Station_point(0,z,R,rho,q,m)

Num_points = 25
Input_data_list_NN = np.zeros((Num_points,2))
Station_points_list = [0 for x in range (0,Num_points)]

for a in range (0,Num_points):
    Ran_x = np.abs(50*np.random.randn())
    p = Station_point(Ran_x,z,R,rho,q,m)
    p.norm_g(p0.g)
    Input_data_list_NN[a][0] = Ran_x
    Input_data_list_NN[a][1] = p.g0
    Station_points_list[a] = p

print(Input_data_list_NN)

[[32.55802208  0.46649978]
 [30.75941366  0.49814544]
 [27.73836822  0.55490381]
 [ 7.78334912  0.94577987]
 [14.89090752  0.82309639]
 [41.471594    0.33457526]
 [17.24017935  0.7744656 ]
 [33.15311371  0.45639883]
 [82.58126292  0.08283955]
 [39.08947901  0.36583267]
 [13.67908368  0.84713061]
 [52.42907167  0.22316428]
 [ 2.14708227  0.99569367]
 [49.8802401   0.24485419]
 [59.73657058  0.17224178]
 [63.62978302  0.15074799]
 [ 7.26681859  0.95246033]
 [54.9017839   0.20419002]
 [41.05562781  0.33983254]
 [14.83156008  0.82429332]
 [51.37588843  0.23185197]
 [63.47164382  0.15155618]
 [35.07307328  0.42508188]
 [61.66165339  0.16118658]
 [27.56375262  0.55831346]]
